In [1]:
!pip install tensorflow


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [3]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


ResNet (Residual Networks) is a deep neural network architecture that has one of the most influential architectures in deep learning due to its innovative use of residual learning, which addresses the problem of vanishing/exploding gradients in very deep networks.

ResNet Architecture:
The typical ResNet architecture looks like this:
1.   Initial layers: Conv layer + BatchNorm + ReLU + MaxPool
2.   Several residual blocks (with skip connections)
3.   Global Average Pooling
4.   Fully connected layer
5.   Softmax (for classification tasks)

Residual : instead of learning f(x), it learns g(x)=f(x) - x
then calculates : g(x)+x


In [4]:
# Freeze the base model (i.e., the convolutional layers)
base_model.trainable = False

In [5]:
model = Sequential([
    base_model,  # Pre-trained ResNet50 as the base
    Flatten(),   # Flatten the 3D output to 1D
    Dense(256, activation='relu'),  # Add a fully connected layer
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Probleme : ResNet50 est tres lourd --> Probleme de RAM

#Donc il faut utiliser un modele plus leger comme ResNet 34 ou MobileNet


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Charger le dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normaliser les valeurs des pixels
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir les labels en one-hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Créer un générateur d'images pour redimensionner à 224x224
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=32
)

# Modèle MobileNetV2 pré-entraîné
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ajouter des couches supplémentaires
model = models.Sequential()
model.add(layers.Input(shape=(32, 32, 3)))  # Ajuster la taille d'entrée
model.add(layers.Lambda(lambda x: tf.image.resize(x, (224, 224))))  # Redimensionner les images
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(10, activation='softmax'))

# Compiler le modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()

# Entraîner le modèle avec le générateur
model.fit(train_generator, epochs=10, validation_data=(x_test, y_test))


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                      │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │          12,810 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 2,236,682 (8.53 MB)

 Non-trainable params: 34,112 (133.25 KB)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 255s 130ms/step - accuracy: 0.6288 - loss: 1.0985 - val_accuracy: 0.1972 - val_loss: 3.4459
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 202s 109ms/step - accuracy: 0.7924 - loss: 0.6079 - val_accuracy: 0.1309 - val_loss: 3.9422
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 201s 108ms/step - accuracy: 0.8229 - loss: 0.5156 - val_accuracy: 0.1101 - val_loss: 3.0901
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 203s 109ms/step - accuracy: 0.8416 - loss: 0.4715 - val_accuracy: 0.1490 - val_loss: 5.5555
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 166s 106ms/step - accuracy: 0.8546 - loss: 0.4243 - val_accuracy: 0.1245 - val_loss: 2.6755
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 167s 107ms/step - accuracy: 0.8665 - loss: 0.3922 - val_accuracy: 0.1432 - val_loss: 2.9263
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 170s 108ms/step - accuracy: 0.8741 - loss: 0.3639 - val_accuracy: 0.1224 - val_loss: 2.6232
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 199s 106ms/step - accuracy: 0.8